Importing Libraries

In [ ]:
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.transform import rowcol
from scipy.stats import mode
import matplotlib.pyplot as plt
from collections import Counter

from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

import tensorflow as tf
from tensorflow.keras import layers, models

Image Coordinates

In [ ]:
points = [
    Point(77.1, 28.7),
    Point(77.5, 28.9),
    Point(78.2, 29.4),
    Point(76.5, 27.9),
    Point(75.0, 26.0)   # outside area
]

images_gdf = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")

Loading ESA Land Cover GeoTIFF

In [ ]:
src = rasterio.open("/Users/kashvimishrasingh/Downloads/Scenario-1/dataset/worldcover_bbox_delhi_ncr_2021.tif")

ESA mapping to 11 classes

In [ ]:
ESA_TO_LABEL = {
    10: 0, 20: 1, 30: 2, 40: 3, 50: 4,
    60: 5, 70: 6, 80: 7, 90: 8, 95: 9, 100: 10
}

Patch Extraction

In [ ]:
def extract_patch(src, lon, lat, size=128):
    r, c = rowcol(src.transform, lon, lat)
    h = size // 2
    win = Window(c - h, r - h, size, size)
    return src.read(1, window=win)

Labeling Assignment

In [ ]:
def get_label(patch, nodata=0):
    valid = patch[patch != nodata]
    if len(valid) == 0:
        return None

    esa = mode(valid, keepdims=False).mode
    return ESA_TO_LABEL.get(esa, None)

Building Dataset

In [ ]:
X, y = [], []

for p in images_gdf.geometry:
    patch = extract_patch(src, p.x, p.y)
    label = get_label(patch)

    if label is None:
        continue

    X.append(patch)
    y.append(label)

X = np.array(X)
y = np.array(y)

Normalization

In [ ]:
X = X / 255.0
X = np.expand_dims(X, -1)
X = np.repeat(X, 3, axis=-1)

Training spliting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Class distribution

In [ ]:
cnt = Counter(y)
plt.bar(cnt.keys(), cnt.values())
plt.xlabel("Class")
plt.ylabel("Count")
plt.title("Class Distribution")
plt.show()

Q3. CNN MODEL 

Architecture

In [ ]:
model = models.Sequential([
    layers.Conv2D(16, 3, activation='relu', padding='same', input_shape=(128,128,3)),
    layers.MaxPooling2D(),

    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(11, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

Model Training

In [ ]:
model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=8,
    validation_split=0.2
)